In [ ]:
import os
import gradio as gr
import requests
import re
from bs4 import BeautifulSoup
from openai import OpenAI
from dotenv import load_dotenv

In [ ]:
load_dotenv(override=True)
openai_api_key= os.getenv('OPENAI_API_KEY')

if openai_api_key:
    print("API Key looks good so far")
else:
    print("API Key does not exist")

openai = OpenAI(api_key=openai_api_key)
MODEL = 'gpt-4o-mini'

In [ ]:
system_message = (
    "You are a helpful AI assistant. "
    "You can analyze websites and summarize their content if provided a URL. "
    "Always respond in clean, well-structured markdown."
)

In [ ]:
def scrape_website(url):
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
    except requests.exceptions.RequestException:
        return None, "⚠️ Failed to fetch website."

    soup = BeautifulSoup(response.content, "html.parser")
    title = soup.title.string.strip() if soup.title else "No title found"

    if soup.body:
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        text = soup.body.get_text(separator="\n", strip=True)
    else:
        text = "No readable content found."

    return title, text

In [ ]:
def chat(message, history):
    """Main chatbot function"""
    messages = [{"role": "system", "content": system_message}] + history

    url_pattern = r"^(https?://[^\s]+)$"
    if re.match(url_pattern, message.strip()):
        # If input is a URL → scrape website
        title, text = scrape_website(message.strip())

        if text.startswith("⚠️ Failed"):
            yield "⚠️ Could not fetch the website. Try another URL."
            return

        # Build a custom GPT prompt to analyze the website
        prompt = f"Analyze and summarize the following website:\n\n**Title:** {title}\n\n{text}"
        messages.append({"role": "user", "content": prompt})
    else:
        # Otherwise, treat as normal chatbot message
        messages.append({"role": "user", "content": message})

    # Stream GPT response
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=messages,
        stream=True
    )

    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result

In [ ]:
gr.ChatInterface(
    fn=chat,
    type="messages",
    title="🤖 AI Chatbot + Web Scraper",
    description="Ask anything OR provide a URL, and I'll fetch & analyze it in real-time!"
).launch()